In [2]:
from bs4 import BeautifulSoup
import requests
from pprint import pprint

In [12]:
url = 'https://jeffhuang.com/best_paper_awards/'
page = requests.get(url)
with open('best-paper.html', 'w', encoding="utf-8") as f:
    f.write(page.text)

In [13]:
with open('best-paper.html', 'r', encoding="utf-8") as f:
    content = f.read()

soup = BeautifulSoup(content, 'html.parser')

tables_full = soup.find_all('table', attrs={'class': 'table'})

def add_to_confs(confs, head, authors):
    if head is not None:
        conf = head.find('a').get_text()
        area = head.find('span').get_text()
        confs[conf] = (area, authors)
    return confs

tables = {}
for t in tables_full:
    year = t.get('id')
    body = t.find('tbody')
    rows = body.find_all('tr')
    confs = {}
    temp_head = None
    temp_authors = []
    for r in rows:
        head = r.find('th', attrs={'class': 'category-name'})
        if head is not None:
            confs = add_to_confs(confs, temp_head, temp_authors)
            temp_head = head
            temp_authors = []
        
        authors_elem = r.find('td', attrs={'class': 'authors'})
        new_authors = []
        
        first_author = authors_elem.find_next(text=True)
        new_authors.append(first_author.strip())
        
        if authors_elem.find('div'):
            authors_div_elem = authors_elem.find('div')
        
            for br_tag in authors_div_elem.find_all('br'):
                br_tag.replace_with('\n')
            new_authors.extend([i.strip() for i in authors_div_elem.get_text().split('\n')])
        
        temp_authors.append([i for i in new_authors if i.strip()])
        
    confs = add_to_confs(confs, temp_head, temp_authors)

    tables[year] = confs

C:\Users\Aneesh\AppData\Local\Temp\ipykernel_17676\3086072952.py:33: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  first_author = authors_elem.find_next(text=True)


In [14]:
rows = []
rows.append('Author|University|Conference|Area|Year|Coauthors')
for year, confs in tables.items():
    for conf, val in confs.items():
        area = val[0]
        authorset = val[1]
        for authors in authorset:
            authornum = len(authors)
            for author in authors:
                author_uni = author.split(",")
                author = author_uni.pop(0).strip()
                uni = ''.join(author_uni).strip()
                row = f"{author}|{uni}|{conf}|{area}|{year}|{authornum}"
                rows.append(row)

In [10]:
for r in rows[0:5]:
    print(r)

Author|University|Conference|Area|Year|Coauthors
Abhishek Bhattacharyya|University of WisconsinâMadison|ISCA| Architecture|2022|3
Abhijith Somashekhar|Apple|ISCA| Architecture|2022|3
Joshua San Miguel|University of WisconsinâMadison|ISCA| Architecture|2022|3
Budmonde Duinkharjav|New York University|SIGGRAPH| Graphics|2022|5


In [17]:
with open('data/best-paper.csv', 'w', encoding="utf-8") as file:
    for item in rows:
        file.write(item + '\n')